In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0, ResNet50V2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.utils.class_weight import compute_class_weight

# Verify GPU availability
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

base_dir = 'C:\\Users\\Θάνος\\Desktop\\Thesis Thanasis\\data_aug_3'
subfolders = ['clear', 'clouds']
categories = ['Healthy_augmented', 'Damaged_augmented']
IMG_HEIGHT = 64
IMG_WIDTH = 64
BATCH_SIZE = 32

def load_data(base_dir, subfolders, categories, img_height, img_width):
    data = []
    labels = []
    image_paths = []
    for category in categories:
        class_num = categories.index(category)
        for subfolder in subfolders:
            folder_path = os.path.join(base_dir, subfolder, category)
            images = sorted(os.listdir(folder_path))
            for img_name in images:
                if img_name.endswith('.png'):
                    img_path = os.path.join(folder_path, img_name)
                    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(img_height, img_width))
                    img_array = tf.keras.preprocessing.image.img_to_array(img)
                    data.append(img_array)
                    labels.append(class_num)
                    image_paths.append((subfolder, category, img_name))
    return np.array(data), np.array(labels), image_paths

data, labels, image_paths = load_data(base_dir, subfolders, categories, IMG_HEIGHT, IMG_WIDTH)
data = data / 255.0

# Split data ensuring twins are in the same split
def split_data(image_paths):
    unique_image_ids = list(set([img_name for subfolder, category, img_name in image_paths]))
    train_ids, test_ids = train_test_split(unique_image_ids, test_size=0.2, random_state=42)
    train_ids, val_ids = train_test_split(train_ids, test_size=0.25, random_state=42)  # 0.25 * 0.8 = 0.2
    return train_ids, val_ids, test_ids

def get_split_indices(image_paths, split_ids):
    split_indices = [i for i, (subfolder, category, img_name) in enumerate(image_paths) if img_name in split_ids]
    return split_indices

train_ids, val_ids, test_ids = split_data(image_paths)
train_indices = get_split_indices(image_paths, train_ids)
val_indices = get_split_indices(image_paths, val_ids)
test_indices = get_split_indices(image_paths, test_ids)

X_train, y_train = data[train_indices], labels[train_indices]
X_val, y_val = data[val_indices], labels[val_indices]
X_test, y_test = data[test_indices], labels[test_indices]

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Validation labels shape: {y_val.shape}")
print(f"Test labels shape: {y_test.shape}")

# Define data augmentation with seed
def create_datagen(seed=None):
    return ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.3,
        height_shift_range=0.3,
        shear_range=0.3,
        zoom_range=0.3,
        horizontal_flip=True,
        fill_mode='nearest'
    ), seed

datagen, seed = create_datagen(seed=42)  # Set the seed for reproducibility
datagen.fit(X_train)
train_generator = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE, seed=seed)  # Use the seed here too
val_generator = datagen.flow(X_val, y_val, batch_size=BATCH_SIZE, seed=seed)

# Compute class weights using the training set
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weights = dict(enumerate(class_weights))

print(f"Class weights: {class_weights}")

# Define model building functions
def build_mobilenetv2_model(input_shape):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(2, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

def build_efficientnetb0_model(input_shape):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(2, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

def build_resnet50v2_model(input_shape):
    base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False
    inputs = Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    outputs = Dense(2, activation='softmax')(x)
    model = Model(inputs, outputs)
    return model

input_shape = (IMG_HEIGHT, IMG_WIDTH, 3)
models = {
    "MobileNetV2": build_mobilenetv2_model(input_shape),
    "EfficientNetB0": build_efficientnetb0_model(input_shape),
    "ResNet50V2": build_resnet50v2_model(input_shape)
}

# Compile models
for name, model in models.items():
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=1)

# Train and evaluate each model
histories = {}
test_results = {}
for name, model in models.items():
    print(f"Training {name} model...")
    history = model.fit(
        train_generator,
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        epochs=200,
        validation_data=val_generator,
        class_weight=class_weights,
        callbacks=[early_stopping, reduce_lr],
        verbose=1
    )
    histories[name] = history

    # Save the fine-tuned model
    model_path = f'best_{name}_model.h5'
    #model.save(model_path)
    print(f"Best {name} fine-tuned model saved to {model_path}")

    # Evaluate the model on the test set
    test_predictions = model.predict(X_test)
    y_test_true = np.argmax(y_test, axis=1)
    y_test_pred = np.argmax(test_predictions, axis=1)

    test_conf_matrix = confusion_matrix(y_test_true, y_test_pred)
    test_class_report = classification_report(y_test_true, y_test_pred, target_names=categories)

    test_results[name] = {
        "confusion_matrix": test_conf_matrix,
        "classification_report": test_class_report
    }

# Print the evaluation results for each model
for name, results in test_results.items():
    print(f"Confusion Matrix (Test) for {name}:")
    print(results["confusion_matrix"])
    print(f"Classification Report (Test) for {name}:")
    print(results["classification_report"])

TensorFlow version: 2.10.0
Num GPUs Available:  1
Training data shape: (7248, 64, 64, 3)
Validation data shape: (2416, 64, 64, 3)
Test data shape: (2416, 64, 64, 3)
Training labels shape: (7248, 2)
Validation labels shape: (2416, 2)
Test labels shape: (2416, 2)
Class weights: {0: 0.8511038046031001, 1: 1.2120401337792641}
94668760/94668760 [==============================] - 12s 0us/step
Training MobileNetV2 model...
Epoch 1/200
226/226 [==============================] - 24s 68ms/step - loss: 0.9279 - accuracy: 0.4877 - val_loss: 0.8010 - val_accuracy: 0.4913 - lr: 1.0000e-04
Epoch 2/200
226/226 [==============================] - 14s 63ms/step - loss: 0.8647 - accuracy: 0.5033 - val_loss: 0.7518 - val_accuracy: 0.5128 - lr: 1.0000e-04
Epoch 3/200
226/226 [==============================] - 15s 66ms/step - loss: 0.8384 - accuracy: 0.4992 - val_loss: 0.7459 - val_accuracy: 0.5116 - lr: 1.0000e-04
Epoch 4/200
226/226 [==============================] - 14s 64ms/step - loss: 0.8028 - accuracy

C:\Anaconda\envs\thesis\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda\envs\thesis\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Anaconda\envs\thesis\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


226/226 [==============================] - 24s 85ms/step - loss: 1.5100 - accuracy: 0.4972 - val_loss: 0.9342 - val_accuracy: 0.5178 - lr: 1.0000e-04
Epoch 2/200
226/226 [==============================] - 17s 76ms/step - loss: 1.1696 - accuracy: 0.5115 - val_loss: 0.8159 - val_accuracy: 0.5327 - lr: 1.0000e-04
Epoch 3/200
226/226 [==============================] - 17s 76ms/step - loss: 1.0508 - accuracy: 0.5140 - val_loss: 0.7622 - val_accuracy: 0.5563 - lr: 1.0000e-04
Epoch 4/200
226/226 [==============================] - 17s 74ms/step - loss: 0.9993 - accuracy: 0.5051 - val_loss: 0.7737 - val_accuracy: 0.5373 - lr: 1.0000e-04
Epoch 5/200
226/226 [==============================] - 17s 76ms/step - loss: 0.9257 - accuracy: 0.5184 - val_loss: 0.7530 - val_accuracy: 0.5344 - lr: 1.0000e-04
Epoch 6/200
226/226 [==============================] - 17s 77ms/step - loss: 0.8650 - accuracy: 0.5223 - val_loss: 0.7335 - val_accuracy: 0.5559 - lr: 1.0000e-04
Epoch 7/200
226/226 [===================